In [2]:
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.table import Table

In [5]:
file = fits.open('pfsObject-20240902-170048-99723d8d89340884.fits')
file2 = fits.open('pfsObject-20240902-170453-a490d17fbc8adc73.fits')

In [6]:
file2.info()

Filename: pfsObject-20240902-170453-a490d17fbc8adc73.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  FLUX          1 ImageHDU        18   (11376,)   float32   
  2  MASK          1 ImageHDU        19   (11376,)   int32   
  3  TARGET        1 BinTableHDU     32   5R x 2C   [5A, E]   
  4  SKY           1 ImageHDU        18   (11376,)   float32   
  5  COVAR         1 ImageHDU        19   (11376, 3)   float32   
  6  COVAR2        1 ImageHDU         8   (1, 1)   float32   
  7  OBSERVATIONS    1 BinTableHDU     24   4R x 7C   [J, 6A, J, J, K, 2E, 2E]   
  8  NOTES         1 BinTableHDU     11   0R x 0C   []   
  9  FLUX_TABLE    1 BinTableHDU     19   12288R x 4C   [D, E, E, K]   


In [23]:
file2[1].header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    1 / number of array dimensions                     
NAXIS1  =                11376                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
WCSAXES =                    1 / Number of coordinate axes                      
CRPIX1  =               5689.5 / Pixel coordinate of reference point            
CDELT1  =                8E-11 / [m] Coordinate increment at reference point    
CUNIT1  = 'm'                  / Units of coordinate increment and value        
CTYPE1  = 'WAVE'               / Vacuum wavelength (linear)                     
CRVAL1  =             8.05E-07 / [m] Coordinate value at reference point        
LATPOLE =                 90

In [8]:
!pwd

/home/amos/SUBARU_PFS


In [2]:
hscdata = pd.read_csv('HSC_simon.csv')
hscdata.drop(columns=['Unnamed: 0'], inplace=True)
hscdata['priority'] = 4

LSBGs = pd.read_csv('LSBGs.csv')
LSBGs['priority'] = 3

miyaji = pd.read_fwf('./miyaji_prior_0.txt', header=0)
miyaji['priority'] = 3
for i, row in miyaji.iterrows():
    miyaji.at[i, 'HSC-ID'] = '1392511090'+ str(i)
    
miyaji1 = pd.read_fwf('./miyaji_prior_1.txt', header=0)
miyaji1['priority'] = 3
for i, row in miyaji1.iterrows():
    miyaji1.at[i, 'HSC-ID'] = '1392511091' + str(i)
miyaji=pd.concat([miyaji, miyaji1], ignore_index=True)
miyaji.rename(columns={'RA':'ra', 'Dec':'dec', 'G-band':'GCm'}, inplace=True)
miyaji['H_a'] = 1.0

nepsource_hz = pd.read_csv('SJ_highz.csv')
nepsource_hz['priority'] = 2

nepsource_lz = pd.read_csv('SJ_lowz.csv')
nepsource_lz['priority'] = 0

cat = pd.concat([nepsource_hz, nepsource_lz], axis=0)
print(len(cat))
cat = pd.concat([cat, LSBGs], axis=0)
print(len(cat))
cat = pd.concat([cat, miyaji], axis=0)
print(len(cat))
cat = pd.concat([cat, hscdata], axis=0)
print(len(cat))
cat = cat[~cat.duplicated(subset='HSC-ID')]
print(len(cat))

12374
14005
14079
275493
272186


In [51]:

print(len(cat), len(cat[cat['priority']==0]))

272186 7612


In [55]:
cat.to_csv('Mergedcat4PFS.csv', index=False)

In [3]:
merger = Table.read('NEP-mergers.fits').to_pandas()
merger['priority'] = 3
merger.rename(columns={'HSC_ID':'HSC-ID'}, inplace=True)

In [4]:
a = pd.concat([merger, cat], axis=0)

In [5]:
a = a[a.duplicated(subset='HSC-ID')]
a[['HSC-ID', 'priority']].to_csv('Merger_id.csv', index=False)

In [15]:
a

,HSC_ID,forced_RA,forced_Dec,spec_z,photo_z,best_z,vis_merger,priority,# Trt-Ptch,HSC-ID,...,L24,L24_err,flag,H_b_L18,H_b_S9,H_b,specz,prior,H_a,r_hsc
1,7.921778e+16,270.488831,65.300400,-99.0,0.2266,0.2266,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.921778e+16,270.429077,65.287163,-99.0,0.2886,0.2886,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.921778e+16,270.398529,65.351166,-99.0,0.2993,0.2993,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.921779e+16,270.490692,65.419792,-99.0,0.2544,0.2544,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7.921779e+16,270.466400,65.396248,-99.0,0.2780,0.2780,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,8.009834e+16,271.121552,67.672836,-99.0,0.2238,0.2238,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2128,8.009834e+16,271.119904,67.680931,-99.0,0.2888,0.2888,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2129,8.009834e+16,271.083038,67.692421,-99.0,0.2671,0.2671,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2130,8.009834e+16,271.054932,67.721596,-99.0,0.2954,0.2954,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:

a[a.duplicated(subset='HSC-ID')]

,HSC_ID,forced_RA,forced_Dec,spec_z,photo_z,best_z,vis_merger,priority,# Trt-Ptch,HSC-ID,...,L24,L24_err,flag,H_b_L18,H_b_S9,H_b,specz,prior,H_a,r_hsc
1,7.921778e+16,270.488831,65.300400,-99.0,0.2266,0.2266,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.921778e+16,270.429077,65.287163,-99.0,0.2886,0.2886,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.921778e+16,270.398529,65.351166,-99.0,0.2993,0.2993,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.921779e+16,270.490692,65.419792,-99.0,0.2544,0.2544,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7.921779e+16,270.466400,65.396248,-99.0,0.2780,0.2780,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,8.009834e+16,271.121552,67.672836,-99.0,0.2238,0.2238,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2128,8.009834e+16,271.119904,67.680931,-99.0,0.2888,0.2888,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2129,8.009834e+16,271.083038,67.692421,-99.0,0.2671,0.2671,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2130,8.009834e+16,271.054932,67.721596,-99.0,0.2954,0.2954,True,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
